# Introduction to DuckDB

[DuckDB](https://duckdb.org/) is an open-source analytical SQL database management system designed for high performance and efficiency. It's built to excel in handling complex analytical queries on large volumes of data while maintaining a lightweight footprint. DuckDB is known for its exceptional speed in executing queries and its ability to operate seamlessly within various environments, from laptops to large-scale server setups.

One of its standout features is its support for standard SQL queries, making it accessible to users familiar with SQL syntax. Additionally, DuckDB is optimized for read-heavy workloads, making it an ideal choice for data exploration, analytics, and research purposes.

DuckDB also provides integration with popular data science tools like Jupyter Notebooks and Pandas, facilitating a smooth workflow for analysts and data scientists. Its compatibility with Jupyter Notebooks allows for an interactive and collaborative environment where users can harness DuckDB's power alongside their code and analysis.

Moreover, DuckDB's integration with Pandas simplifies data manipulation and analysis. It enables the execution of SQL queries directly on Pandas DataFrames, providing a familiar interface for those comfortable with Pandas while leveraging DuckDB's speed and efficiency for data processing.

Another notable aspect is that it can handle various data formats such as CSV, Parquet, and others. And let's you interact with these formats through SQL queries streamlines the process of querying and analyzing diverse data sources without the need for extensive data preprocessing.

So DuckDB offers a user-friendly environment for data exploration, analysis, and manipulation.

To get started let's import duckdb and pandas. To be able to convert jupyter notebook cells to sql cells we need to also load the jupysql extension.

In [ ]:
import duckdb
import pandas as pd
import json

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

### Configure the notebook
Set configrations on the ipython sql extension to directly output data to Pandas and to simplify the output that is printed to the notebook.

In [ ]:
%config SqlMagic.autopandAS = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

### Connect to a DuckDB database
Connect to a DuckDB Database using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, or a file backed db.

mention that we can also connect to a database in the cloud eg

In [ ]:
%sql duckdb:///duckdb_db/sampleDB.duckdb

## Loading data

We have two files in our data directory:
- pokemon__donations.json
- pokemon__masterdata.csv

### Load a json file into duckdb

So we will see how we can use DuckDB on json and csv files. let's start with the json. Here we will create a pandas dataframe first and than load that into the database.

In [ ]:
df_donations = pd.read_json('data/pokemon__donations.json', orient='records')
df_donations.head()

With duckdb we now can query this dataframe using SQL.

In [ ]:
%%sql

Select * from df_donations
where donation_amount_eur > 1000;

Or we can quite easily create a table from this dataframe in a duckdb database with duckdb:

In [ ]:
%%sql
CREATE TABLE pokemon__donations AS SELECT * FROM df_donations

### Load a csv file into duckdb

But we can also load a csv file directly into duckdb. 

In [ ]:
%%sql

SELECT * FROM 'data/pokemon__masterdata.csv';

And of course we also can create a table in the duckdb database. We can do this with the COPY command. First we create the table:

In [ ]:
%%sql

CREATE TABLE pokemon__masterdata(
    pokemon_identifier INT,
    pokemon_name VARCHAR,
    pokemon_type VARCHAR
    );

And than we can load the data into the table:

In [ ]:
%%sql

COPY pokemon__masterdata FROM 'data/pokemon__masterdata.csv' (HEADER TRUE, DELIMITER ',');

## Querying the database and exporting the results

So now we created the tables and loaded the data into the database we can query the database using SQL. Let's start with the donations table:

In [ ]:
%%sql

select * from pokemon__donations

In [ ]:
%%sql

select * from pokemon__masterdata

We could now join the donations table with the pokemon table to get the pokemon names for the donations. And export the result to a pandas dataframe:

In [ ]:
%%sql

df_join << SELECT * FROM pokemon__donations 
LEFT JOIN pokemon__masterdata 
ON pokemon__donations.pokemon_id = pokemon__masterdata.pokemon_identifier

In [ ]:
df_join = df_join.DataFrame()
df_join.head()

Or we could save the result to a csv file:

In [ ]:
%%sql

COPY(
    SELECT * FROM pokemon__donations 
    LEFT JOIN pokemon__masterdata 
    ON pokemon__donations.pokemon_id = pokemon__masterdata.pokemon_identifier
)
TO 'data/output.csv' (HEADER, DELIMITER ',');

## Why should you use DuckDB over Pandas? 

You shouldn't (except you like SQL more than Python). But you can use DuckDB in combination with Pandas. So you can combine the power of DuckDB and the power of Pandas.